In [ ]:
%matplotlib qt
#qt
#from matplotlib import interactive
#interactive(True)
from func003_GUImatplotlib import gui,gui2

#input("Press Enter to continue...")
import sys
import os
import numpy as np
import scipy as sp
import scipy.optimize as opt
import math as math
import matplotlib.pyplot as plt
import itertools as IT
from scipy.integrate import simps
from func000_Plotset import plot_ini
from func000_Plotset import plotset
plot_ini()
plt.rcParams["figure.figsize"] = [9, 8]
##############################################################################
#iはファイル番号に対応　i=1>A001.cdat
fil_i=list(range(20,22,1)) # a1以上a2未満までを2ステップで作成　b = list(range(a1, a2, 2))

#BG_art=0.5e19 #Artificially decrease BG line
x_ini=[488.1,489.4]#fitting range
PeakPosi_ini=[488.69,488.85,488.4,489.15,489.2]
PeakBunsan_ini=[0.05,0.05,0.05,0.05,0.05]
PeakHeight_ini=[1.3e19,1.6e18,1e18,1e18,1e18]
BG_ini=[1.0e15,0.0]
#fitting_ini=PeakPosi+PeakBunsan+PeakHeight+BG

pathOUT="W-Re_fitting.dat"
flib2="../../../../lib/Re-I-WRe.sdat"
lib2=flib2
iteration=100
pltmax=2.0e20

nx=200
number_of_peaks=len(PeakPosi_ini)
nn=number_of_peaks

b1=(-np.inf, np.inf)
boundPosi=(b1,)*number_of_peaks
boundBunsan=((0,0.1),)*number_of_peaks
boundHeight=((0,np.inf),)*number_of_peaks

x_min,x_max=x_ini
##############################################################################
datadir="20180918/ME5000"
i=1
fil_a=[""]*99
while i<99:
    if i<10:
        fil_a[i-1]='cdat/A_00{0:}.cdat'.format(i)
    if i>9:
        fil_a[i-1]='cdat/A_0{0:}.cdat'.format(i)
    i += 1
fil='cdat/A_00{0:}.cdat'.format(i)

file_list=fil_a
print(fil_i)

os.chdir("C:/Users/Ibano/Anaconda3/jupyter/LIBS")
print("initial directory")
init_dir=os.getcwd()
print(os.getcwd())
os.chdir(datadir)

if not os.path.exists('python'):
    os.makedirs('python')

loop_dir=os.getcwd()
k=-1
#for file1 in file_list:
for i in fil_i:
    k+=1
    
    file1=file_list[i-1]
    
    x_min,x_max=x_ini
    XXX, YYY=0.0e0,0.0e0

    print("loop number={}, dir?".format(i))
    print(os.getcwd())
    print("file=",file1)
    with open(file1, 'rb') as file:
        SPEC = np.loadtxt(file, skiprows=12,delimiter=None,  unpack=True)
#    print(SPEC)
#    SPEC=np.c_(SPEC)
    if not os.path.exists('python'):
        os.makedirs('python')    
    if not os.path.exists('python/MechelleSpect_00{0:}'.format(i)):
        os.makedirs('python/MechelleSpect_00{0:}'.format(i))

    os.chdir('python/MechelleSpect_00{0:}'.format(i))
    print('python/MechelleSpect_00{0:}'.format(i))
    XX=SPEC[0]
    YY=SPEC[3]
    ######################################    
    #plt.plot(XX,YY);
    #plt.xlabel('Wavelengh[nm]');
    #plt.ylabel('Intensity [cps]');
    #plt.savefig("1-1_Spectral.png",format = 'png', dpi=300)
    #plt.close()
    ######################################

    j=0
    
    while XX[j] < x_min:
        jmin=j
        j+=1
    
    while XX[j] < x_max:
        jmax=j
        j+=1
        
    jlen=jmax-jmin
    print("jmin={0:},jmax={1:}".format(jmin,jmax))
    print("XX(jmin)={0:},XX(jmax)={1:}".format(XX[jmin],XX[jmax]))
    XXX=XX[jmin:jmax]
    YYY=YY[jmin:jmax]
    
    ###BG assumption###################### 
    def axb(x1,x2,y1,y2): #y=ax+b
        a=(y2-y1)/(x2-x1)
        b=y1-a*x1
        return a,b
    BGt1,BG=axb(XXX[0],XXX[jlen-1],YYY[0],YYY[jlen-1])
    #BG1=YYY[jlen-1]
    #BG2=YYY[0]
    #BGt1=(BG1-BG2)/(XXX[jlen-1]-XXX[0])
    #BG=YYY[0]-BGt1*XXX[0]#-BG_art
    BGt=[BGt1,BG]
    ###################################### 
    ###GUI FITTING########################
    if k==0:        
        ###READING TEMPFILE#########################################
        #flg=0
        def yes_no_input():
            choice = input("Use previous data? 'yes' or 'no' [y/N]: ").lower()
            if choice in ['y', 'ye', 'yes','']:
                return True
            elif choice in ['n', 'no']:
                return False

        
        if yes_no_input():
            with open("../temp.dat", 'rb') as file:
                p2 = np.loadtxt(file, delimiter=",",  unpack=True)
            PeakPosi_ini,PeakBunsan_ini,PeakHeight_ini,BGt=p2[0:nn],p2[nn:2*nn],p2[2*nn:3*nn],p2[3*nn:3*nn+2]
        #############################################################
        ppp=gui(XXX,YYY,PeakPosi_ini,PeakBunsan_ini,PeakHeight_ini,BGt,nn)
        XXX,YYY,PeakPosi_ini,PeakBunsan_ini,PeakHeight_ini,BG_ini,nn = ppp
        ###MAKING TEMPFILE###########################################
        ppp2=list(PeakPosi_ini)+list(PeakBunsan_ini)+list(PeakHeight_ini)+list(BG_ini)
        ppp2=np.c_[ppp2]
        with open("../temp.dat", mode='wb') as f:
            np.savetxt(f,ppp2, delimiter=', ')
        #with open("../temp.dat", mode='w') as ftemp:
        #    ftemp.write("{}".format(ppp2),delimiter=",") #ftemp.write("{}".format(list(ppp2)))
        #############################################################
    m=PeakPosi_ini
    sd=PeakBunsan_ini
    a=PeakHeight_ini
    BGt=BG_ini
    boundPosi=list(boundPosi)
    jb=0
    while jb<nn:
        boundPosi[jb]=(m[jb]*0.99998,m[jb]*1.00002)
        jb+=1
    
    ######################################    
 #   plt.plot(XXX,YYY);
 #   plt.xlim(x_min,x_max);
 #   plt.ylim(0,1.e21);
 #   plt.xlabel('Wavelengh[nm]');
 #   plt.ylabel('Intensity [cps]');
 #   plt.savefig("1-2_Spectral-zoom.png",format = 'png', dpi=300)
    ######################################

    x = np.arange(x_min, x_max, (x_max-x_min)/nx)

    def norm(x, mean, sd):
      norm = []
      for i in range(x.size):
        norm += [1.0/(sd*np.sqrt(2*np.pi))*np.exp(-(x[i] - mean)**2/(2*sd**2))]
      return np.array(norm)
############################################################################
    # Solving
    p = list(m)+list(sd)+list(a)+list(BGt) # Initial guesses for leastsq
    m,sd,a,BGt=p[0:nn],p[nn:2*nn],p[2*nn:3*nn],p[3*nn:3*nn+2]
    k=0
    y_init=0.0

    BGline_ini=BGt[0]*x+BGt[1]

    while k < nn:
        y_init += a[k]*norm(x,m[k],sd[k])
        k +=1 
    y_init +=BGt[0]*x+BGt[1]

     
    m,sd,a,BGt=list(PeakPosi_ini),list(PeakBunsan_ini),list(PeakHeight_ini),list(BGt)
    nn=number_of_peaks
    
    def res(p, x, y):     
      m,sd,a,BGt=p[0:nn],p[nn:2*nn],p[2*nn:3*nn],p[3*nn:3*nn+2]
      k=0
      y_fit=0.0
      while k < nn:
        y_fit += a[k]*norm(x,m[k],sd[k])
        k +=1 
      y_fit+=BGt[0]*x+BGt[1]
      err = y - y_fit
      return err
    pBG=1e-3
    bBG1=(BGt[0]*(1.0-np.sign(BGt[0])*pBG),BGt[0]*(1.0+np.sign(BGt[0])*pBG))
    bBG2=(BGt[1]*(1.0-np.sign(BGt[1])*pBG),BGt[1]*(1.0+np.sign(BGt[1])*pBG))
    boundBG=(bBG1,bBG2)#(-np.inf,YYY[jlen-1]))
    bounds=tuple(boundPosi)+boundBunsan+boundHeight+boundBG #fitting constrains
    bounds=np.c_[bounds]
    
    plsq = opt.least_squares(res, p, bounds=bounds,args = (XXX,YYY)) #method : {‘trf’, 'dogbox’, ‘lm’},
    #m,sd,a,BGt=p[0:nn],p[nn:2*nn],p[2*nn:3*nn],p[3*nn:3*nn+2]
    j=0
    
    while j<nn*3:
        p = list(m)+list(sd)+list(a)
        p[0:nn]=plsq.x[0:nn]
        p[nn:2*nn]=plsq.x[nn:2*nn]
        p[2*nn:3*nn]=plsq.x[2*nn:3*nn]
        p[3*nn:3*nn+2]=plsq.x[3*nn:3*nn+2]
        jk=0
        while jk<j:
            bounds[0,jk]=p[jk]
            bounds[1,jk]=p[jk]*(1.0+np.sign(p[jk])*1e-15)
            jk+=1

        plsq = opt.least_squares(res, p, bounds=bounds,args = (XXX,YYY)) #method : {‘trf’, 'dogbox’, ‘lm’}, 
        j+=1
############################################################################
    #PeakPosi,PeakBunsan,PeakHeight,BGa,BG = plsq.x[:]
    PeakPosi=plsq.x[0:nn]
    PeakBunsan=plsq.x[nn:2*nn]
    PeakHeight=plsq.x[2*nn:3*nn]
    BGa,BG=plsq.x[3*nn:3*nn+2]
    
    m1,m2=PeakPosi[0],PeakPosi[1]
    sd1,sd2=PeakBunsan[0],PeakBunsan[1]
    a1,a2=PeakHeight[0],PeakHeight[1]
    if nn>2:
        m3,sd3,a3=PeakPosi[2],PeakBunsan[2],PeakHeight[2]
    if nn>3:
        m4,sd4,a4=PeakPosi[3],PeakBunsan[3],PeakHeight[3]
    if nn>4:
        m5,sd5,a5=PeakPosi[4],PeakBunsan[4],PeakHeight[4]
        
    BGline=BGa*x+BG
    y1_est= a1*norm(x, m1, sd1)+BGline
    y2_est= a2*norm(x, m2, sd2)+BGline
    if nn>2:
        y3_est= a3*norm(x, m3, sd3)+BGline
    if nn>3:
        y4_est= a4*norm(x, m4, sd4)+BGline
    if nn>4:
        y5_est= a5*norm(x, m5, sd5)+BGline
        
    y_est = y1_est+y2_est-BGline
    if nn>2:
        y_est = y_est+y3_est-BGline
    if nn>3:
        y_est = y_est+y4_est-BGline
    if nn>4:
        y_est = y_est+y5_est-BGline
        
    eqint1=simps(a1*norm(x, m1, sd1),x)
    eqint2=simps(a2*norm(x, m2, sd2),x)
    
    prt1='m1={0:.2f},m2={1:.2f}'.format(m1, m2)
    prt2='sd1={0:.2f},sd2={1:.2f}'.format(sd1,sd2)
    prt3='a1={0:.2e},a2={1:.2e}'.format(a1,a2)
    prt4='BGa={0:.2e},BG={1:.2e}'.format(BGa,BG)
    prt5='eqint1={0:.2e},eqint2={1:.2e} '.format(eqint1,eqint2)
    
    fig = plt.figure()  
    plt.title(prt1+"\n"+prt2+"\n"+prt3+"\n"+prt4+"\n"+prt5,fontsize=8)
    
    plt.plot(XXX, YYY,'ro', label='Real Data')
    plt.plot(x, y_init, lw=1, label='Starting Guess')
    plt.plot(x, y_est, 'y-', lw=3,label='y_tot_Fitted')
    plt.plot(x, y1_est, 'b-',lw=1, label='y1_Fitted')
    plt.plot(x, y2_est, 'g-',lw=1, label='y2_Fitted')
    if nn>2:
        plt.plot(x, y3_est, lw=1, label='y3_Fitted')
    if nn>3:
        plt.plot(x, y4_est, lw=1, label='y4_Fitted')
    if nn>4:
        plt.plot(x, y5_est, lw=1, label='y5_Fitted')
    
     
    plt.text(0.1, 0.95, file1, transform=fig.transFigure)    
    plt.plot(x, BGline, label='BG')
    plt.xlim(x_min,x_max);
    plt.ylim(0,pltmax);
    plotset()
    plt.legend(loc='upper right',fontsize=12) #bbox_to_anchor=(1.8,1)
    plt.savefig("1-3_Spectral-fit.png",format = 'png', dpi=300)
    plt.show(block=True)
    plt.close()
    
    ####Writing data########################################################

    with open(lib2, 'rb') as lib:
        FL = np.genfromtxt(lib,delimiter=None,skip_header=4,skip_footer=5)
        
    LibWavelength,LibWave1,LibWave2 = FL[0],FL[1],FL[2]
    Aki,Fik,Ei = FL[3],FL[4],FL[5]
    Ek,Gi,Gk = FL[6],FL[7],FL[8]
    print(LibWave1)
    x_peak=m2
    x_min=m2-sd2*10
    x_max=m2+sd2*10
    Integral=eqint2
    IperGA=np.log(Integral/Gk/Aki)
    output=np.c_[x_peak,x_min,x_max,Aki,Fik, Ei,Ek,Gi,Gk,Integral,IperGA]
    with open(pathOUT, mode='w') as f:
        f.write("x_peak,x_min,x_max,Aki,Fik, Ei,Ek,Gi,Gk,Integral,Ln(IperGA) \n")
    #output=np.c_[m1, m2, sd1, sd2, a1, a2, BGa,BG,eqint1,eqint2]
    with open(pathOUT, mode='ab') as f:
        np.savetxt(f,output,fmt='%.4e', delimiter=', ') #
    ########################################################################
############################################################################    
    os.chdir(loop_dir)
############################################################################
os.chdir(init_dir)


[20, 21]
initial directory
C:\Users\Ibano\Anaconda3\jupyter\LIBS
loop number=20, dir?
C:\Users\Ibano\Anaconda3\jupyter\LIBS\20180918\ME5000
file= cdat/A_020.cdat
python/MechelleSpect_0020
jmin=14256,jmax=14298
XX(jmin)=488.09717,XX(jmax)=489.37015
Use previous data? 'yes' or 'no' [y/N]: 
